# FunctionTree tutorial: using VAMPyR to run calculations
##### *Path described for Linux Subsystem on Windows 10 software



## Installation of VAMPyR: 

- On Ubuntu terminal, type:

        ~ git clone https://github.com/MRChemSoft/vampyr.git
        ~ cd vampyr
        ~ ./setup
        ~ cd build
        ~ make

## Using VAMPyR through Jupyter Notebook:

- Install Xming and execute it
- On Ubuntu terminal, type:

        ~ export DISPLAY=:0
        ~ jupyter notebook

## Steps on using VAMPyR's FunctionTree:

    1. Importing libraries that shall be used to run the code 
    2. Defining calculation's basis parameters
    3. Choose and create mathematic operator 
    4. Creating FunctionTree and applying operator
    5. FunctionTree's functions

### 1. Importing libraries that shall be used to run the code 

In this example, VAMPyR is imported as "vp" 

In [1]:
import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt

### 2. Defining calculation's basis parameters

For definitions and access to default input values, please visit the webpage:

- https://mrchem.readthedocs.io/en/latest/mrchem_manual.html#top-section

In [2]:
# Alters the lower bounds to -(2**min_scale) and the upper bounds to +(2**min_scale)
min_scale = -4   
# Precision of calculations up until defined decimal place
prec = 1e-4

# Multiplies de unit lengh and drags the lower bounds to "corner"*"unit length"
corner = [-1, -1, -1]
boxes = [2, 2, 2]
# Defines how large the calculation space will be
world = vp.BoundingBox(min_scale, corner, boxes)

# Number of basis functions
order = 5
# To select desired basis (LegendreBasis or InterpolatingBasis polynomials)
basis = vp.InterpolatingBasis(order)
# Uses defined variables to create the MultiResolutionAnalysis function 
MRA = vp.MultiResolutionAnalysis(world, basis, 25)


# Define the function to be calculated
def f(x):
    return x[0]**2

### 3. Choose and create mathematic operator(s) 

To have it detailed how many and which arguments are required by each operator, please run the line:

- help(vp.NAME_OF_OPERATOR)

The following operators are currently implemented in VAMPyR:

##### Convolution operators
The convolution operators will adaptively build the output tree based on the chosen precision (note that there areseparate precision parameters for the construction and application of convolution operators).

In [3]:
# Creates a convolution operator called "IC" with a narrow Gaussian kernel, close to Dirac’s deltafunction.
IC = vp.IdentityConvolution(MRA, 0.01)

# Creates a convolution operator with the Poisson Green’s function kernel called "PO" that runs with defined precision.
PO = vp.PoissonOperator(MRA, prec)

# Creates a convolution operator called "HO" with the complex Helmholtz Green’s function kernel.
HO = vp.HelmholtzOperator(MRA, 0.0, prec)

##### Derivative operators
The derivatives operator have clearly defined requirements on the output grid structure, based on the grid of the inputfunction. This means that there is no real grid adaptivity, and thus no precision parameter is needed for the applicationof such an operator.

In [4]:
# Creates the Alpert, Beylkin, Gines, Vozovoi derivative operator called "AO".
AO = vp.ABGVOperator(MRA, 0.1, 0.1)

# Creates a derivative operator called "PH" based on Pavel Holoborodko’s smoothing derivative.
PH = vp.PHOperator(MRA, 1)

# Creates a derivative operator based on projection onto B-splines, a smoothingderivative operator, called "BS". 
BS = vp.BSOperator(MRA, 1)

### 4. Creating FunctionTree and applying operator

Depending on the type of operator chosen (Derivative or Convolution), the "apply" agurments should vary. Please run < help(vp.apply) > for more detailed instructions.

In [5]:
# Creates a Function Tree called "f_tree" in MRA basis 
f_tree = vp.FunctionTree(MRA)

# Creates a Function Tree called "d_f_tree" in MRA basis 
d_f_tree = vp.FunctionTree(MRA)

# Defines Project: (precision defined "prec"; FunctionTree "f_tree"; function that will be inserted in f_tree "f")
vp.project(prec, f_tree, f)

# Applies designed operator (IC, PO, HO, AO, PH, BS) on a FunctionTree; 
# (precision defined "prec"; output FunctionTree "d_f_Tree"; ABGVOperator 
# "AO"; FunctionTree in which the operator will be applied "f_Tree")
vp.apply(prec, d_f_tree, IC, f_tree, 1)

### 5. FunctionTree's functions

To have it detailed which and how many arguments are required by each object, please run the line:

- help(FunctionTree.NAME_OF_OBJECT)

The following objects can be applied on FunctionTree function in order to compute specified evaluation;

In [ ]:
# .evalf = Returns the function value in a given point. Possibly inaccurate, once only the scaling part of 
# the leaf nodes will be evaluated
d_f_tree.evalf([0.0, 0.0, 0.0])

# .clear = Clear MW coefficients and remove all grid refinement
d_f_tree.clear()

# .integrate = Returns the integral of the function over the entire computational domain.
d_f_tree.integrate()

# .normalize = Rescales the function by its norm
d_f_tree.normalize()

# .getEndValues = Returns the ending wavelength values
d_f_tree.getEndValues([0,0])

# .setEndValues = Sets ending wavelength values
d_f_tree.setEndValues([0,0])

# .saveTree = Saves tree to file
d_f_tree.saveTree(a)

# .loadTree = Loads tree to file
d_f_tree.loadTree(a)

# .crop = Truncate the wavelet expansion accoring to a new precision threshold
b = 10
d_f_tree.crop(5.0, 1.0, b<10)

# .multiply = Multiply the function by a given number and it is multiplied by the function
d_f_tree.multiply(2, d_f_tree)

# .power = Raise an existing function to a given power
d_f_tree.power(2.0)

# .square = Multiply an existing function with itself
d_f_tree.square()

# .rescales = Multiply the function with a scalar, fixed grid
d_f_tree.rescale(0.9)


In [ ]:
help(vp.AnalyticFunction)